### RAGAS


In [ ]:
from dotenv import load_dotenv
import os
# load .env file to environment
load_dotenv()

OPEN_API_KEY = os.getenv("OPENAI_API_KEY")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
QDRANT_URL = os.getenv("QDRANT_URL")


In [1]:
from langchain.document_loaders import JSONLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient

# Load JSON data
loader = JSONLoader(
    file_path="careers.json",
    jq_schema=".[] | {job_name: .job_name, description: .description, spec: .spec}",
    text_content=False  # We extract structured fields
)
documents = loader.load()

In [4]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o", api_key=OPEN_API_KEY))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/tmp/ipykernel_676097/3817641369.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())


In [19]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
golden_dataset = generator.generate_with_langchain_docs(documents, testset_size=5)

Generating Samples: 100%|██████████| 4/4 [00:06<00:00,  1.55s/it]


In [ ]:
from langchain_qdrant import QdrantVectorStore

client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
embedding_dim = 1536

jobs_vector_store = QdrantVectorStore(
    client=client,
    collection_name="job_definitions",
    embedding=embedding_model,
)
jobs_retriever = jobs_vector_store.as_retriever(search_kwargs={"k":4})


In [10]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

In [11]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

In [13]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | jobs_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [14]:
naive_retrieval_chain.invoke({"question" : "Give me a job relatd to web development"})["response"].content

'A job related to web development is "Développeur Full Stack." This role involves designing and developing complete web applications from front-end to back-end using technologies like JavaScript, Node.js, React, Vue.js, Python, SQL, and NoSQL.'

In [21]:
for test_row in golden_dataset:
  response = naive_retrieval_chain.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"].content
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

dataset=golden_dataset.to_pandas()

In [22]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset)

In [23]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

In [26]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating: 100%|██████████| 24/24 [05:28<00:00, 13.69s/it]


{'context_recall': 1.0000, 'faithfulness': 0.3184, 'factual_correctness': 0.2650, 'answer_relevancy': 0.9656, 'context_entity_recall': 0.3854, 'noise_sensitivity_relevant': 0.1462}

In [28]:
result

{'context_recall': 1.0000, 'faithfulness': 0.3184, 'factual_correctness': 0.2650, 'answer_relevancy': 0.9656, 'context_entity_recall': 0.3854, 'noise_sensitivity_relevant': 0.1462}

In [31]:
result.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,context_recall,faithfulness,factual_correctness,answer_relevancy,context_entity_recall,noise_sensitivity_relevant
0,What role do APIs play in the responsibilities...,"[{""job_name"": ""D\u00e9veloppeur Backend"", ""des...","[{""job_name"": ""D\u00e9veloppeur Backend"", ""des...",The role of APIs in the responsibilities of a ...,A Backend Developer is often in charge of the ...,1.0,0.333333,0.33,0.990865,0.125000,0.000000
1,What role does CI/CD play in the responsibilit...,"[{""job_name"": ""Ing\u00e9nieur DevOps"", ""descri...","[{""job_name"": ""Ing\u00e9nieur DevOps"", ""descri...","CI/CD, which stands for Continuous Integration...",A DevOps Engineer uses CI/CD tools to automate...,1.0,0.222222,0.46,1.000000,0.000000,0.000000
2,Azure do what in cloud?,"[{""job_name"": ""Ing\u00e9nieur en Cloud Computi...","[{""job_name"": ""Ing\u00e9nieur en Cloud Computi...",Azure is a cloud computing platform that provi...,L’ingénieur Cloud déploie et gère les infrastr...,1.0,0.333333,0.17,0.903922,0.833333,0.200000
3,In the context of cloud infrastructure managem...,"[{""job_name"": ""Ing\u00e9nieur DevOps"", ""descri...","[<1-hop>\n\n{""job_name"": ""Ing\u00e9nieur en Cl...",DevOps practices facilitate collaboration betw...,DevOps practices play a crucial role in facili...,1.0,0.384615,0.10,0.967590,0.583333,0.384615
